In [1]:
import pandas as pd
import numpy as np
import random
from pulp import *
import math

## Data Preparation

### Helper functions to structure data into other formats


In [2]:
def get_fuel_cost(p):
    annual_fuel_cost = 0
    for k in p.keys():
        details = k.split('_')
        annual_fuel_cost = annual_fuel_cost + consumption['_'.join(details[1:4]) + '_' + details[7]]*\
        fuel_cost[details[5] + '_' + details[7]]*veh_range['_'.join(details[1:4])] * p[k] 
    return annual_fuel_cost

def get_emissions_cost(p):
    annual_emissions = []
    dbg = []
    for y in years[0:5]:
        emit = 0        
        #x_dbg = 0
        for k in p.keys():
            details = k.split('_')
            if details[2] in fuel_adj.keys():
                adj = fuel_adj[details[2]][details[6]]
            else:
                adj = 1
            if details[5] == str(y):
                emit += consumption['_'.join(details[1:4]) + '_' + details[7]]*\
                fuel_emissions[details[5] + '_' + details[7]]*veh_range['_'.join(details[1:4])] * p[k] * adj
                if emit < 0:
                    print("negative emissions")
                
        annual_emissions.append(emit)
    return annual_emissions

In [3]:
def get_veh_cost(p):
    total_veh_cost = 0
    for k in p.keys():
        details = k.split('_')
        total_veh_cost = total_veh_cost + vehicle_cost['_'.join(details[1:5])] * p[k]
    return total_veh_cost

In [4]:
# demand
def apply_demand_constraint(linear_program, availability, size):
    for y in years:
        for demand_bucket in ['D1', 'D2', 'D3', 'D4']:
            df = availability[(availability['demand_year'] == y) & (availability['Size'] == size) & 
                              (availability['demand_distance'] == demand_bucket)]
            if df.shape[0] == 0:
                print(y, size, demand_bucket)
                return df
            specific_demand = df['Demand (km)'].iloc[0]
            constraint = []
            for _id, yr_range, u, s, f in zip(df['ID'], df['Yearly range (km)'], 
                                              df['usage_distance'], df['sell_year'], df['Fuel']):
                constraint.append(usage_variables['_'.join([_id, str(s), str(y), u, f])] * yr_range)
            linear_program += lpSum(x for x in constraint) >= specific_demand 
    return linear_program

In [5]:

def create_use_df(fuel_plan):
    year = []
    vid = []
    num_veh = []
    _fuel = []
    dbucket = []
    vdistance = []
    for y in years:
        for v in fuel_plan.keys():
            d = v.split('_')
            if d[5] == str(y):
                year.append(y)
                vid.append('_'.join(d[1:4]))
                num_veh.append(fuel_plan[v])
                _fuel.append(d[7])
                dbucket.append(d[6])
                vdistance.append(veh_range['_'.join(d[1:4])])
    use_df = pd.DataFrame({'Year': year,
                           'ID': vid,
                           'Num_Vehicles': num_veh,
                           'Type': 'Use',
                           'Fuel': _fuel,
                           'Distance_bucket': dbucket,
                           'Distance_per_vehicle(km)': vdistance}).sort_values('Year')#.to_csv('submission.csv')
    return use_df

In [6]:
def apply_limited_sales_constraint(linear_program, except_pct, other_pct):
    # limited sales
    for y in years:
        sold_vehicles = []
        in_service = []

        for v in purchase_objective.keys():
            details = v.split('_')
            if int(details[3]) == y:
                sold_vehicles.append(v)
            elif (int(details[3]) > y) & (int(details[2]) <= y):
                in_service.append(v)
        if y in except_pct.keys():
            linear_program += lpSum(purchase_variables[x] for x in sold_vehicles)*(1-except_pct[y])\
            <= lpSum(purchase_variables[x] for x in in_service)*except_pct[y]
        else:
            linear_program += lpSum(purchase_variables[x] for x in sold_vehicles)*(1-other_pct) <= lpSum(purchase_variables[x] for x in in_service)*other_pct
    return linear_program

In [7]:
def create_buy_sell_dfs(purchase_plan):
    # buy and sell df
    buy_year = []
    sell_year = []
    vid = []
    n_veh = []
    for p in purchase_plan.keys():
        details = p.split('_')
        buy_year.append(details[3])
        sell_year.append(details[4])
        n_veh.append(purchase_plan[p])
        vid.append('_'.join(details[1:4]))

    buy_df = pd.DataFrame({'Year': [int(y) for y in buy_year],
                           'ID': vid,
                           'Num_Vehicles': n_veh})

    sell_df = pd.DataFrame({'Year': [int(y) for y in sell_year],
                           'ID': vid,
                           'Num_Vehicles': n_veh})
    sell_df = sell_df[sell_df['Year'] < 2039]

    buy_df = buy_df.groupby(['Year', 'ID'], as_index=False).sum()
    sell_df = sell_df.groupby(['Year', 'ID'], as_index=False).sum()
    buy_df['Type'] = 'Buy'
    buy_df['Distance_per_vehicle(km)'] = 0
    sell_df['Type'] = 'Sell'
    sell_df['Distance_per_vehicle(km)'] = 0
    return buy_df, sell_df


In [8]:
def pct_of_inventory(buy_df, sell_df):
    buy_sell = buy_df[['Year', 'Num_Vehicles']].groupby(['Year'], as_index=False).sum().merge(sell_df[['Year', 'Num_Vehicles']].groupby(['Year'], as_index=False).sum(), on='Year') 
    running_total = 0
    running_column = []
    running_pct = []
    for b, s in zip(buy_sell['Num_Vehicles_x'], buy_sell['Num_Vehicles_y']):
        running_pct.append(s/(running_total+b))
        running_total = running_total + b - s # end of year
        running_column.append(running_total)
    buy_sell['running_total'] = running_column 
    buy_sell['pct'] = running_pct
    return buy_sell

In [9]:
def smart_pick(dictionary, k):
    if k in dictionary:
        return dictionary[k]
    else:
        return 0
    
def vehicle_running_total(buy_df, sell_df, use_df, dbg=None):
    dfs = []
    df = pd.DataFrame({'Year':years})
    
    for v, n in zip(buy_df['ID'], buy_df['Num_Vehicles']):
        
        bdf = buy_df[(buy_df['ID'] == v)].copy()              # this should just be 1
        sdf = sell_df[(sell_df['ID'] == v)].copy()           
        
        edf = use_df[(use_df['ID'] == v)].copy()
        edf = edf[['ID', 'Year', 'Num_Vehicles']].groupby(['ID', 'Year'], as_index=False).sum()
        
        buy_dict = {}
        sell_dict = {}
        expected_dict = {}
        for y, n in zip(bdf['Year'],bdf['Num_Vehicles']):
            buy_dict[y] = n
        for y, n in zip(sdf['Year'],sdf['Num_Vehicles']):
            sell_dict[y] = n
        for y, n in zip(edf['Year'],edf['Num_Vehicles']):
            expected_dict[y] = n

        running_total = 0
        buy = []
        sell = []
        available_beg = []
        available_end = []
        expected = []
        difference = []
        for y in years:
            
            buy_amt = smart_pick(buy_dict, y)
            buy.append(buy_amt)
            sell_amt = smart_pick(sell_dict, y)
            sell.append(sell_amt)
            expected_amt = smart_pick(expected_dict, y)
            expected.append(expected_amt)
            difference.append(running_total + buy_amt - expected_amt)
            available_beg.append(running_total + buy_amt)
            running_total += buy_amt - sell_amt
            available_end.append(running_total)
            
        if dbg:
            if dbg == v:
                return pd.DataFrame({'Year': years, v+'_beg': available_beg, v+'_end': available_end,
                                     'buy': buy, 'sell': sell, v+'_xpd':expected})
                
        df = df.merge(pd.DataFrame({'Year': years, v: difference}), on='Year')
        
        df.set_index('Year', inplace=True)
        
    return df.T
            #    buy_total = buy_df[['ID', 'Year', 'Num_Vehicles']].groupby(['ID', 'Year'], as_index=False).sum()
    

In [10]:
def available_check(available, purchase_plan, dbg=None):
    models = []
    needed_years = []
    needed_amt = []
    new_purchase_plan2 = purchase_plan.copy()
    for m, inventory in zip(available.index, available.to_numpy()):

        needed = [y for y, inv in zip(years, inventory) if inv < 0 ]
        amt = [abs(inv) for y, inv in zip(years, inventory) if inv < 0 ]
        if len(needed) > 0:
            print('need {}'.format(m))

            for y, a in zip(needed,amt):        
                models.append(m)
                needed_years.append(y)
                needed_amt.append(a)
                options = []
                for p in new_purchase_plan2.keys():
                    d = p.split('_')
                    d2 = m.split('_')
                    if '_'.join(d[1:4]) == '_'.join(d2[0:3]):
                        if int(d[4]) >= y:
                            options.append(p)
                            print("considering {} {} {} {}".format(p, d[3], d[4], y))

                if len(options) == 0:
                    print('this is a bigger problem')
                    print(y, a, m)
                new_vehicle = options[random.randint(0, len(options)-1)]
                new_purchase_plan2[new_vehicle] += a
                print(new_vehicle, new_purchase_plan2[new_vehicle], a)

    return pd.DataFrame({'ID': models, 'Years': needed_years, 'Amt': needed_amt}), new_purchase_plan2

In [11]:
def update_fuel_adj(verify_demand):
    # use miles driven instead of range for fuel
    fuel_adj = {}
    for size in range(4):
        fuel_adj['S' + str(size+1)] = {}
    for s, d, dmnd, cpcty in zip(verify_demand['Size'], verify_demand['Distance'], verify_demand['Demand (km)'], 
                                 verify_demand['capacity']):
        fuel_adj[s][d] = dmnd/cpcty
    return fuel_adj

### Import Data

In [12]:
path = 'c:\\users\\mark113\\desktop\\jupyter\\shell\\'
demand = pd.read_csv(path + 'dataset\\demand.csv')
fuels = pd.read_csv(path + 'dataset\\fuels.csv')
cost_profiles = pd.read_csv(path + 'dataset\\cost_profiles.csv')
vehicles = pd.read_csv(path + 'dataset\\vehicles.csv')
vehicles_fuels = pd.read_csv(path + 'dataset\\vehicles_fuels.csv')
carbon_df = pd.read_csv(path + 'dataset\\carbon_emissions.csv')

In [13]:
max_year = 2038
years = [y for y in range(2023, max_year+1)]
max_ownership = 10

In [14]:
emissions_limit = {}
for e, y in zip(carbon_df['Carbon emission CO2/kg'], carbon_df['Year']):
    emissions_limit[y] = e

In [15]:
# build some dictionaries for easier reference
fuel_adj = {}

consumption = {}
for v, f, c in zip(vehicles_fuels['ID'], vehicles_fuels['Fuel'], vehicles_fuels['Consumption (unit_fuel/km)']):
    consumption[v+'_'+f] = c
fuel_cost = {}
fuel_emissions = {}
for f, y, e, c in zip(fuels['Fuel'], fuels['Year'], fuels['Emissions (CO2/unit_fuel)'], fuels['Cost ($/unit_fuel)']):
    fuel_cost[str(y)+'_'+f] = c
    fuel_emissions[str(y)+'_'+f] = e     
veh_range = {}
for v, r in zip(vehicles['ID'], vehicles['Yearly range (km)']):
    veh_range[v] = r

In [16]:
vehicle_usage = []
for d in ['D1', 'D2', 'D3', 'D4']:
    for u in range(int(d[1])):
        for year in years:
            for y in range(max_ownership):
                df = vehicles[(vehicles['Distance']==d) & (vehicles['Year'] == year)].copy()
                if df.shape[0] > 0:
                    df['usage_distance'] = 'D' + str(u+1)
                    df['sell_year'] = df['Year'] + y
                    vehicle_usage.append(df.copy())
        
V = pd.concat(vehicle_usage)
V = V[V['sell_year'] <= (max_year+1)]
V.head(1000).tail()


,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance,usage_distance,sell_year
179,LNG_S3_2026,LNG,S3,2026,165150,73000,D4,D1,2031
67,Diesel_S1_2026,Diesel,S1,2026,92881,102000,D4,D1,2032
83,Diesel_S2_2026,Diesel,S2,2026,113643,106000,D4,D1,2032
99,Diesel_S4_2026,Diesel,S4,2026,165299,118000,D4,D1,2032
115,Diesel_S3_2026,Diesel,S3,2026,129140,73000,D4,D1,2032


In [17]:
vehicles_df = V.merge(vehicles_fuels[['ID', 'Fuel']], on='ID')

In [18]:
ownership_costs = [.06, .09, .12, .15, .18, .21, .24, .27, .3, .33]
accum_own_cost = [np.sum(ownership_costs[0:(x+1)]) for x in range(max_ownership)]
resale_cost = [.9, .8, .7, .6, .5, .4, .3, .3, .3, .3]

In [19]:
# 1. calculate vehcile cost
# 2. join demand to vehicles based on what's available for the demand year

vehicles_df.columns = ['ID', 'Vehicle', 'Size', 'vehicle_year', 'Cost ($)', 'Yearly range (km)', 'vehicle_distance', 
                       'usage_distance', 'sell_year', 'Fuel']
vehicle_cost = {}
for v, cost in zip(vehicles['ID'], vehicles['Cost ($)']):
    for yr_sold in range(max_ownership):
        if (int(v[-4:]) + yr_sold) > max_year:
            resale = 0
        else:
            resale = resale_cost[yr_sold]
        vehicle_cost[v + '_' + str(int(v[-4:])+yr_sold)] = cost *(1+accum_own_cost[yr_sold] - resale)
availability = demand.merge(vehicles_df, left_on=['Size', 'Distance'], right_on=['Size', 'usage_distance'])  
availability.columns = ['demand_year', 'Size', 'demand_distance', 'Demand (km)', 'ID', 'Vehicle', 'vehicle_year', 
                        'Cost ($)', 'Yearly range (km)', 'vehicle_distance', 'usage_distance', 'sell_year', 'Fuel']
availability = availability[(availability['vehicle_year'] <= availability['demand_year']) & 
                            ((availability['sell_year']) >= availability['demand_year'])] 

In [20]:
veh_fuel_combo = availability.copy()
veh_fuel_combo['distance_integer'] = [int(vd[1]) for vd in veh_fuel_combo['vehicle_distance']]
veh_fuel_combo = veh_fuel_combo.drop(['usage_distance', 'demand_distance', 'vehicle_distance', 'Demand (km)'], axis=1).drop_duplicates()
veh_fuel_combo.shape

(11900, 10)

### Check for "dominance"
For the same buy, sell year, size, distance group
one vehicle has better fuel cost, fuel emissions for every usage year and better
vehicle cost than another, then it completely dominates that vehicle

In [ ]:

dominated_id = []
dominated_sell_yr = []
dominated_fuel = []
for beg_yr in years:
    for end_yr in [beg_yr+1, beg_yr+2]:
        for db in ['D1', 'D2', 'D3', 'D4']:
            for sb in ['S1', 'S2', 'S3', 'S4']:
                vid_for_df = []
                sell_year_for_df = []
                fuel_for_df = []
                fuel_value = []
                e_value = []
                v_cost_for_df = []
                vd_df = []
                df = veh_fuel_combo[(veh_fuel_combo['vehicle_year']==beg_yr) & (veh_fuel_combo['sell_year']==end_yr) & 
                                    (veh_fuel_combo['Size']==sb)]
                for v in df['ID'].unique():
                    single_df = df[df['ID']==v]

                    for f in single_df['Fuel'].unique():
                        v_cost_for_df.append(vehicle_cost[v+'_'+str(end_yr)])
                        single_fuel_df = single_df[single_df['Fuel'] == f]
                        vid_for_df.append(v)
                        sell_year_for_df.append(end_yr)

                        fcost = sum([fuel_cost[str(dy)+'_'+f]*consumption[v+'_'+f]*veh_range[v]
                                     for dy in single_fuel_df['demand_year']])
                        ecost = sum([fuel_emissions[str(dy)+'_'+f]*consumption[v+'_'+f] for dy in single_fuel_df['demand_year']])
                        fuel_for_df.append(f)
                        fuel_value.append(fcost)
                        e_value.append(ecost)
                        vd_df.append(single_fuel_df['distance_integer'].iloc[0])


                comparison = pd.DataFrame({'ID': vid_for_df, 'fuel_value': fuel_value, 'emissions_value': e_value, 
                                           'sell_year': sell_year_for_df, 'vehicle_cost': v_cost_for_df,
                                          'idist': vd_df, 'fuel': fuel_for_df})
                comparison['total_cost'] = [x+y for x, y in zip(comparison['fuel_value'], comparison['vehicle_cost'])]

                for a in range(comparison.shape[0]-1):
                    for b in range(a+1, comparison.shape[0]):
                        v1, fv1, e1, s1, vc1, d1, f1, tc1 = comparison.iloc[a]
                        v2, fv2, e2, s2, vc2, d2, f2, tc2 = comparison.iloc[b]
                        if (e1 >= e2) & (tc1 >= tc2) & (d1 <= d2):
                            print("1 {} {} {} {}".format(v1, v2, f1, f2))
                            dominated_id.append(v1)
                            dominated_sell_yr.append(end_yr)
                            dominated_fuel.append(f1)
                        elif (e1 <= e2) & (tc1 <= tc2) & (d1 >= d2):
                            print("2 {} {} {} {}".format(v1, v2, f1, f2))
                            if (v2 == 'LNG_S4_2032') & (f1 == 'Electricity') & (f2 == 'LNG'):
                                ref_df = comparison
                            dominated_id.append(v2)
                            dominated_sell_yr.append(end_yr)
                            dominated_fuel.append(f2)
                            


In [22]:
dominated_df = pd.DataFrame({'ID': dominated_id, 'sell_year': dominated_sell_yr, 'Fuel': dominated_fuel})
dominated_df['dominated'] = 1
dominant_availability = availability.merge(dominated_df.drop_duplicates(), how='left', on=['ID', 'sell_year', 'Fuel'])
print(dominant_availability.shape)
dominant_availability = dominant_availability[dominant_availability['dominated'].isna()]
print(dominant_availability.shape)

(43688, 14)
(40388, 14)


### Create variable names
- Purchase variables: ID, purchase year, sell year
- Usage variables: ID, purchase year, sell year, size, distance category, fuel type, demand year
- Group usage variables by purchase variables

In [23]:
usage_variable_names = []
purchase_objective = {}
fuel_consumption_objective = {}

for v in vehicles['ID']:
    df_v = dominant_availability[dominant_availability['ID'] == v]

    for s in df_v['sell_year'].unique():
        df = df_v[df_v['sell_year'] == s]

        for dy in df['demand_year'].unique():
            
            df_y = df[df['demand_year'] == dy]
            purchase_group =\
            ['_'.join([v, str(s), str(dy), ud, f]) for ud, f in zip(df_y['usage_distance'], df_y['Fuel'])]
            usage_variable_names = usage_variable_names + purchase_group
            if v+'_'+str(s) in purchase_objective.keys():
                purchase_objective[v+'_'+str(s)].append(purchase_group)
            else:
                purchase_objective[v+'_'+str(s)] = [purchase_group]
    for d in df_v['demand_year'].unique():
        for f in df_v['Fuel'].unique():
            df = df_v[(df_v['demand_year'] == d) & (df_v['Fuel'] == f)]
            fuel_consumption_objective[v+'_'+str(d)+'_'+f] =\
            ['_'.join([v, str(s), str(d), u, f])  for s, u in zip(df['sell_year'], df['usage_distance'])]

len(usage_variable_names)            

40388

### Objective Function 

In [24]:
total_cost = LpProblem("Shell", LpMinimize)
usage_variables = pulp.LpVariable.dicts("usage", (_id for _id in usage_variable_names ), lowBound=0, cat="Integer")
purchase_variables = pulp.LpVariable.dicts("purchase", (_id for _id in purchase_objective.keys()) , lowBound=0, cat="Integer")

#### Minimize Fuel cost

In [25]:
objective = []
ct = 0
emissions_constraint = []
for k in fuel_consumption_objective.keys():
    details = k.split('_') # vehicle, size, purchase year, usage year, fuel
    usage_objective = [usage_variables[v]*fuel_cost[details[3]+'_'+details[4]]*\
                       consumption['_'.join(details[0:3])+'_'+details[4]]*\
                       veh_range['_'.join(details[0:3])]  for v in fuel_consumption_objective[k]]
    objective = objective + usage_objective

#### Minimize Full Ownership

In [26]:
objective = objective + [purchase_variables[v]*vehicle_cost[v]  for v in list(purchase_objective.keys())]
total_cost += lpSum(x for x in objective)

### Constraints

#### Demand Constraint
The amount used must be less than the amount purchased

In [27]:
for v in purchase_objective.keys():
    for by_demand_year in purchase_objective[v]:
        total_cost += lpSum(usage_variables[x] for x in by_demand_year) <= purchase_variables[v]

#### Emissions constraint

In [28]:
for y in years:
    
    annual_emissions = []
    emissions_id = {}
    consumption_id = {}
    veh_range_id = {}

    for u in usage_variables.keys():
        d = u.split('_')
        if d[4] == str(y):
            annual_emissions.append(u)
            emissions_id[u] = d[4]+'_'+d[6]
            consumption_id[u] = '_'.join(d[0:3])+'_'+d[6]
            veh_range_id[u] = '_'.join(d[0:3])

    total_cost += lpSum(usage_variables[v]*fuel_emissions[emissions_id[v]]*\
                        consumption[consumption_id[v]]*\
                        veh_range[veh_range_id[v]]  for v in annual_emissions) <= emissions_limit[y]

#### Fuel Risk Constraint

In [29]:
fuel_risk_constraint = []
for k in fuel_consumption_objective.keys():
    details = k.split('_') # vehicle, size, purchase year, usage year, fuel
    usage_objective = [usage_variables[v]*fuel_cost[details[3]+'_'+details[4]]*\
                       consumption['_'.join(details[0:3])+'_'+details[4]]*\
                       veh_range['_'.join(details[0:3])]  for v in fuel_consumption_objective[k]]
    objective = objective + usage_objective

In [33]:
print(total_cost.solve(PULP_CBC_CMD(timeLimit=14400, keepFiles=False, warmStart=True)))
print(LpStatus[total_cost.status])

1
Optimal


In [ ]:
saved = total_cost.writeMPS('total_cost.mps')